In [ ]:
import os.path as pa
from sfft.EasySparsePacket import Easy_SparsePacket
# sfft version: 1.4.0+

# Note: ForceConv="AUTO" (only depends on FWHM of two images) does not always guarantee the best direction.
#       This notebook is a better (but slower) solution to find the optimal convolution direction in terms of convariance level on difference images.
#       For a transient survey, a emperical rule is more practical (see https://arxiv.org/abs/2508.10155).

# configuration: computing backend and resourse
BACKEND_4SUBTRACT = 'Cupy'      # FIXME {'Cupy', 'Numpy'}, Use Numpy if you only have CPUs
CUDA_DEVICE_4SUBTRACT = '0'     # FIXME ONLY work for backend Cupy
NUM_CPU_THREADS_4SUBTRACT = 8   # FIXME ONLY work for backend Numpy

# configuration: how to subtract 
KerHWRatio = 2.0                # FIXME Ratio of kernel half-width to FWHM (typically, 1.5-2.5).
KerPolyOrder = 2                # FIXME {0, 1, 2, 3}, Polynomial degree of kernel spatial variation.
BGPolyOrder = 0                 # FIXME {0, 1, 2, 3}, Polynomial degree of differential background spatial variation.
                                #       it is trivial here, as sparse-flavor-sfft requires sky subtraction in advance.
ConstPhotRatio = True           # FIXME Constant photometric ratio between images?

# configuration: required info in FITS header
GAIN_KEY = 'GAIN'               # NOTE Keyword of Gain in FITS header
SATUR_KEY = 'ESATUR'            # NOTE Keyword of Effective Saturation in FITS header
                                # Remarks: one may think 'SATURATE' is a more common keyword name for saturation level.
                                #          However, note that Sparse-Flavor SFFT requires sky-subtracted images as inputs, 
                                #          we need to use the 'effective' saturation level after the sky-subtraction. 
                                #          e.g., set ESATURA = SATURATE - (SKY + 10*SKYSIG)

# run sfft subtraction
CDIR = pa.dirname(pa.abspath(__file__))
FITS_REF = CDIR + '/input_data/c4d_180806_052738_ooi_i_v1.S20.skysub.resampled.fits'
FITS_SCI = CDIR + '/input_data/c4d_180802_062754_ooi_i_v1.S20.skysub.fits'

PixA_DIFF_ConvR, PixA_DIFF_ConvS = None, None
for ForceConv in ["REF", "SCI"]:
    FITS_DIFF = CDIR + '/output_data/%s.sfftdiff_Conv%s.fits' %(pa.basename(FITS_SCI)[:-5], ForceConv)    # difference
    _PixA_DIFF, SFFTPrepDict = Easy_SparsePacket.ESP(FITS_REF=FITS_REF, FITS_SCI=FITS_SCI, FITS_DIFF=FITS_DIFF, FITS_Solution=None, \
        ForceConv=ForceConv, GKerHW=None, KerHWRatio=KerHWRatio, KerHWLimit=(2, 20), KerPolyOrder=KerPolyOrder, \
        BGPolyOrder=BGPolyOrder, ConstPhotRatio=ConstPhotRatio, MaskSatContam=False, GAIN_KEY=GAIN_KEY, \
        SATUR_KEY=SATUR_KEY, BACK_TYPE='MANUAL', BACK_VALUE=0.0, BACK_SIZE=64, BACK_FILTERSIZE=3, \
        DETECT_THRESH=2.0, DETECT_MINAREA=5, DETECT_MAXAREA=0, DEBLEND_MINCONT=0.005, BACKPHOTO_TYPE='LOCAL', \
        ONLY_FLAGS=[0], BoundarySIZE=30, XY_PriorSelect=None, Hough_MINFR=0.1, Hough_PeakClip=0.7, BeltHW=0.2, \
        PointSource_MINELLIP=0.3, MatchTol=None, MatchTolFactor=3.0, COARSE_VAR_REJECTION=True, CVREJ_MAGD_THRESH=0.12, \
        ELABO_VAR_REJECTION=True, EVREJ_RATIO_THREH=5.0, EVREJ_SAFE_MAGDEV=0.04, StarExt_iter=4, \
        XY_PriorBan=None, PostAnomalyCheck=False, PAC_RATIO_THRESH=5.0, BACKEND_4SUBTRACT=BACKEND_4SUBTRACT, \
        CUDA_DEVICE_4SUBTRACT=CUDA_DEVICE_4SUBTRACT, NUM_CPU_THREADS_4SUBTRACT=NUM_CPU_THREADS_4SUBTRACT, VERBOSE_LEVEL=2)[:2]
    if ForceConv == "REF": PixA_DIFF_ConvR = _PixA_DIFF
    else: PixA_DIFF_ConvS = _PixA_DIFF
    print('MeLOn CheckPoint: TEST FOR SPARSE-FLAVOR-SFFT SUBTRACTION DONE!\n')


In [ ]:
import warnings
import numpy as np
from scipy.spatial import cKDTree
from astropy.convolution import convolve, convolve_fft
from sfft.utils.NeighboringPixelCovariance import NeighboringPixel_Covariance

class Sampling_Background:
    @staticmethod
    def SB(MASK_BKG, HW=35, NUM_SAMP=256, RandomSeed=None, USE_FFT=True):

        # * MeLOn Notes
        #   @ Extract pixel-boxes on background as samples
        #     METHOD: Consider a pixel-box centred at pixel (r, c), It can be a sample of background 
        #     iff the box is empty, where pixels are all undetected. This criteria is equivalent to 
        #     performing a convolution on OBJmask by a all-unity kernel (box-size), then all  
        #     available box-centers are just the zero-value pixels on the convolved image.
        #     P.S. DETECT_MASK can be produced by SEx with CHECKIMAGE_TYPE='OBJECTS'. 
        
        EKER = np.ones((2*HW+1, 2*HW+1))
        DETECT_MASK = ~MASK_BKG
        if np.sum(DETECT_MASK) > 0:
            if USE_FFT:
                PixA_convd = convolve_fft(
                    DETECT_MASK.astype(int), EKER, \
                    boundary='fill', fill_value=0, normalize_kernel=False
                )
            else:
                PixA_convd = convolve(
                    DETECT_MASK.astype(int), EKER, \
                    boundary='fill', fill_value=0, normalize_kernel=False
                )
            CMASK = PixA_convd == 0
        else: CMASK = np.ones(MASK_BKG.shape, dtype=bool)

        # give all available box-centers (considering boundaries)
        CMASK[:HW, :] = False
        CMASK[-HW:, :] = False
        CMASK[:, :HW] = False
        CMASK[:, -HW:] = False
        
        # randomly select box-centers
        if RandomSeed is not None: np.random.seed(RandomSeed)
        RIDX = np.random.choice(np.arange(np.sum(CMASK)), int(1.5*NUM_SAMP))   # redundant
        R, C = np.where(CMASK)
        RC_pit = np.array([R[RIDX], C[RIDX]]).T
        
        # eliminate the overlapping cases
        tol = np.sqrt(2*HW**2)
        Tree = cKDTree(RC_pit+0.5)
        IDX = Tree.query(RC_pit+0.5, k=2, distance_upper_bound=tol)[1][:, 1]
        Avmask = IDX == RC_pit.shape[0]
        RC_pit = RC_pit[Avmask][:NUM_SAMP]
        
        return RC_pit

# identify the difference with low pixel correlation as the canonical one
def meaure_covlevel(PixA_DIFF, MASK_BKG, HW=35, NUM_SAMP=128, RandomSeed=10086, USE_FFT=True):
    RC_pit = Sampling_Background.SB(MASK_BKG=MASK_BKG, HW=HW, NUM_SAMP=NUM_SAMP, RandomSeed=RandomSeed, USE_FFT=USE_FFT)
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        COVL = []
        for r, c in RC_pit:
            im = PixA_DIFF[r-HW: r+HW+1, c-HW: c+HW+1]
            L = NeighboringPixel_Covariance.NPC(PixA_obj=im)[1]
            COVL.append(L)
        COVL = np.nanmean(COVL)
    return COVL

SFFTLmap = SFFTPrepDict['SFFT-LabelMap']
MASK_BKG = SFFTLmap == 0

FITS_SCI[:-5] + f".sfftdiff_Conv{ForceConv}.fits"
COVL_ConvR = meaure_covlevel(PixA_DIFF=PixA_DIFF_ConvR, MASK_BKG=MASK_BKG)
COVL_ConvS = meaure_covlevel(PixA_DIFF=PixA_DIFF_ConvS, MASK_BKG=MASK_BKG)

FITS_DIFF = CDIR + '/output_data/%s.sfftdiff_OptConv.fits' %(pa.basename(FITS_SCI)[:-5])    # best difference
print('MeLOn CheckPoint: Covariance Level Convolve-REF / Convolve-SCI = [%.3f / %.3f]' %(COVL_ConvR, COVL_ConvS))
if COVL_ConvR < COVL_ConvS:
    os.system('ln -s %s %s' %(FITS_DIFF_ConvR, FITS_DIFF))
    print('MeLOn CheckPoint: Convolve-REF is better, your best diff image would be [sfftdiff_ConvREF.fits]')
else:
    os.system('ln -s %s %s' %(FITS_DIFF_ConvS, FITS_DIFF))
    print('MeLOn CheckPoint: Convolve-SCI is better, your best diff image would be [sfftdiff_ConvSCI.fits]')
